# Installation procedure

### Pathos

To install Pathos for the first time on your machine, run:

In [ ]:
!pip install pathos

### Ray

To install Ray for the first time on your machine, run: 

In [ ]:
!pip install -U "ray[default]"

To obtain the local ip node, run:

In [ ]:
!ray start --head --port=6379 --redis-password="cbgt2"

Use the local ip node obtained by running the command line above and run: 

In [ ]:
!ray start --address='local ip node:6379' --redis-password='cbgt2' 

If it doesn't work, shut down the ray server and start it again.
If some ray processes are still opened, run:

In [ ]:
!ray stop --force 

### autopep8

To install autopep8 for the first time on your machine, run: 

In [ ]:
!pip install --upgrade autopep8

Uncomment and run this on any code files you create before pushing to github to prevent indentation problems: 

In [ ]:
#!autopep8 --in-place --aggressive --aggressive <filename>.py

# Compile the main simulator code using cython 

In [ ]:
!python setup.py build_ext --inplace

# Import all the relevant files 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import seaborn as sns
import pathos.multiprocessing

In [ ]:
#Importing scripts:

#Import relevant frames:
import cbgt as cbgt
import pipeline_creation as pl_creat

#Import plotting functions:
import plotting_functions as plt_func
import plotting_helper_functions as plt_help
import postprocessing_helpers as post_help

importlib.reload(plt_help)
importlib.reload(plt_func)
importlib.reload(post_help)

# Choose the experiment and create the main pipeline

Choose the experiment to run, and define the number of choices, the number of simulations/thread to run and number of cores:

In [ ]:
#Choose the experiment:
experiment_choice = 'stop-signal'
number_of_choices = 2

#Define how many simulations to run. 
#Each simulation is executed as a separate thread.
num_sims = 5
num_cores = 7


#Call choose_pipeline with the pipeline object:
pl_creat.choose_pipeline(experiment_choice)
#Create the main pipeline:
pl = pl_creat.create_main_pipeline(runloop=True)

#Set a seed:
seed = np.random.randint(0,99999999,1)[0]
print(seed)

Define data and figures directories:

In [ ]:
data_dir = "./Data/"
figure_dir = "./Figures/"

# Running the pipeline

### Define configuration parameter

In [ ]:
#See init_params.py / generateepochs.py / popconstruct.py / qvalues.py
#to know the parameters contained in each of the following variables 

configuration = {
    'experimentchoice': experiment_choice,
    'inter_trial_interval': None,
    'thalamic_threshold': 30.,
    'movement_time': ['mean', 250], #['constant', 300], #default sampled from N(250,1.5), ["constant",250], ["mean",250]
    'choice_timeout': 300,          #default = 1000
    
    'params': None,  #neuron parameters (init_params.py)
    'pops': None,    #population parameters (init_params.py)
    'receps' : None, #receptor parameters (init_params.py)
    'base' : None,   #baseline stimulation parameters (init_params.py)
    'dpmns' : None,  #dopamine related parameters (init_params.py)
    'd1' : None,     #D1-MSNs population related parameters (init_params.py)
    'd2' : None,     #D2-MSNs population related parameters (init_params.py)
    'channels' : pd.DataFrame([['left'], ['right']], columns=['action']), #action channels related parameters (init_params.py)
    'number_of_choices': number_of_choices,
    'newpathways' : None,     #connectivity parameters (popconstruct.py)
    'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
    'Q_df_set': pd.DataFrame([[0.5, 0.5]],columns=["left", "right"]), #pd.DataFrame([[0.3,0.7]],columns=['left','right']), #initialized Q-values df (qvalues.py)  
    'n_trials': 3,            #number of trials (generateepochs.py)
    'volatility': [None,"exact"], #frequency of changepoints (generateepochs.py)
    'conflict': (1.0, 0),         #probability of the preferred choice (generateepochs.py)
    'reward_mu': 1,               #mean for the magnitude of the reward (generateepochs.py)
    'reward_std': 0.1,            #std for the magnitude of the reward (generateepochs.py)
    'maxstim': 0.8,               #amplitude of the cortical input over base line #0.85
    'sustainedfraction': 0.75,
    
    #Stop 1
    'stop_signal_present': [True,True],
    'stop_signal_probability': [1., 1.],  #[[1,3],[1,2]], #probability of trials that will get the stop-signal / list of trial numbers
    'stop_signal_amplitude': [0.6, 0.6],  #amplitude of the stop signal over base line
    'stop_signal_onset': [30.,60.],       #in ms
    'stop_signal_duration' : ["phase 0",165.], 
    'stop_signal_channel': ["left","left"], #"all" (all channels are given the stop signal) 
                                            #"any" (channel given the stop signal is chosen randomly)
                                            # [list of channels] == subset of channels given the stop signal
    'stop_signal_population':["STN","GPeA"],
    
    'record_variables':["stop_input"],
    
    
    #Opto
    'opt_signal_present': [False],
    'opt_signal_probability': [1.], #[0,1,2,3,4], # probability of trials that will get the optogenetic signal / list of trial numbers
    'opt_signal_amplitude': [.7],   #amplitude of the stop signal over base line
    'opt_signal_onset': [30.],      #in ms
    'opt_signal_duration': [150.],
    'opt_signal_channel': ["all"],  # "all" (all channels are given the stop signal) 
                                    #"any" (channel given the stop signal is chosen randomly)
                                    # [list of channels] == subset of channels given the stop signal
    
    'opt_signal_population':["iSPN"],
}

### Run the simulation

ExecutionManager class can take for 'use': 

- 'none', that corresponds to the singlethreaded mode;
- 'pathos', that corresponds to python's multiprocessing mode;
- 'ray', that corresponds to a multiprocessing library for python that operates on a client-server mode.

The default value is None (singlethreaded mode).

In [ ]:
results = cbgt.ExecutionManager(cores=7, use='ray').run([pl]*num_sims,[configuration]*num_sims)  

# Results

List all the agent variables accessible: 

In [ ]:
results[0].keys()

In [ ]:
results[0]['stop_list_trials_list']

In [ ]:
results[0]['meaneff_GABA']

In [ ]:
results[0]['stop_signal_amplitude']

In [ ]:
experiment_choice

Extract all the relevant dataframes:

In [ ]:
firing_rates, rt_dist = plt_help.extract_relevant_frames(results,seed,experiment_choice)

In [ ]:
results[0]['popfreqs']

In [ ]:
 firing_rates[0]

In [ ]:
recorded_variables = post_help.extract_recording_variables(results,results[0]['record_variables'],seed)

In [ ]:
recorded_variables['stop_input']#.melt(id_vars='Time(ms)')

Plot the recorded variable extracted. Figure is saved in the figure_dir previously specified:

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,4))
sns.set(style="white", font_scale=1.5)
sns.lineplot(x="Time(ms)",y="value",data=recorded_variables['stop_input'],hue='nuclei',lw=3.0,ax=ax)
ax.legend_.remove()
ax.spines.top.set_visible(False)
ax.spines.right.set_visible(False)
plt.tight_layout()
fig.savefig(figure_dir+'stop_input.png')

Extract the data tables from the agent:

In [ ]:
datatables = cbgt.collateVariable(results,'datatables')
datatables[0]

Save the selected variables of results in the data_dir previously specified:

In [ ]:
cbgt.saveResults(results,data_dir+'network_data_'+str(seed)+'.pickle',['popfreqs','popdata','datatables'])

Plot the firing rates extracted (figure handles are returned in fig_handles).
Figure is saved in the figure_dir previously specified:

In [ ]:
FR_fig_handles = plt_func.plot_fr(firing_rates,datatables,results,experiment_choice,True)
FR_fig_handles[0].savefig(figure_dir+"Example_FR_stopsignal.png",dpi=300)